In [1]:
import sys
sys.path.append('..')

from src.autoresmaker.main import AutoResMaker
from src.autoresevaluator.main import AutoResEvaluator

In [2]:
arm = AutoResMaker(
    #llm_name = 'google',
    llm_name = 'openai',
    base_model='cnn',
    dataset_name='cifar10',
    )

In [3]:
elemental_method1 = """
<ELEMENTAL_METHOD name="Momentum Calculation and Application">
This method involves computing the moving average of the gradients (also known as the first moment) and using this to update the model parameters. It's a technique used to accelerate gradients vectors in the right directions, thus leading to faster converging. It is widely used in optimization algorithms, including as a component of the Adam optimizer.

<PYTHON>
def momentum_update(lr, beta1, params, objective, weight_decay, epsilon):
    m = 0  # first moment
    theta = params
    while True:
        g = np.gradient(objective(theta), theta)  # gradient of objective function
        if weight_decay != 0:
            g += weight_decay * theta
        m = beta1 * m + (1 - beta1) * g
        theta -= lr * m
        yield theta
</PYTHON>
</ELEMENTAL_METHOD>
"""

elemental_method2 = """
<ELEMENTAL_METHOD name="RMSprop Calculation and Application">
This method involves computing the moving average of the squared gradients (also known as the second moment), applying bias correction, and using this to adjust the parameter updates. It's designed to adapt the learning rate for each parameter, helping to mitigate the vanishing or exploding gradient problem. This technique is a core part of the Adam optimizer.

<PYTHON>
def rmsprop_update(lr, beta2, params, objective, weight_decay, epsilon):
    v = 0  # second moment
    theta = params
    while True:
        g = np.gradient(objective(theta), theta)  # gradient of objective function
        if weight_decay != 0:
            g += weight_decay * theta
        v = beta2 * v + (1 - beta2) * g**2
        v_hat = v / (1 - beta2)
        theta -= lr * g / (np.sqrt(v_hat) + epsilon)
        yield theta
</PYTHON>
</ELEMENTAL_METHOD>
"""

In [4]:
code = arm.exec(elemental_method1, elemental_method2)

In [5]:
print(code)


import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def train_with_adam(trainloader, params, device):
    net = Net()
    net.to(device)  # Move the model to the specified device

    criterion = nn.CrossEntropyLoss()

    # Adam parameters
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-8
    lr = params['lr']
    weight_decay = params.get('weight_decay', 0)

    # Initialize 

In [ ]:
params = {
    'learning_rate': {'type': 'log_float', 'args': [1e-8, 0.1]},
    'iterations': {'type': 'log_float', 'args': [100, 1000]},
    "lambda": 1
}

In [ ]:
are = AutoResEvaluator(
    task_type='binary classification',
    dataset_name='titanic',
    model_path='/Users/tanakatouma/vscode/autores-evaluator/test/logistic_regression.py',
    #model_path='/Users/tanakatouma/vscode/autores-evaluator/test/improve_logistic_regression.py',
    params=params,
    valuation_index='roc_auc'
    )

In [ ]:
are.exec()